## Unit 4 行为 Part 2解决方案

<img src="../img/robotignite_logo_text.png"/>

## 索引: 

* <a href="#SolutionExercise4-13">Exercise 4.13解决方案</a>
* <a href="#SolutionExercise4-14">Exercise 4.14解决方案</a>

## Exercise 4.13 解决方案<p id="SolutionExercise4-13"></p>

<p style="background:#EE9023;color:white;">**Exercise 4.13**</p>

对于该练习，我们假设所使用功能包的名称为**exercise_413**, 启动文件名为 **move_drone_square.launch**, Python文件名为 **move_drone_square.py**。

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: move_drone_square.launch** </p>

In [ ]:
<launch>
    <node pkg="exercise_413" type="move_drone_square.py" name="move_drone_square" output="screen" />
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: move_drone_square.launch** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: move_drone_square.py** </p>

In [ ]:
#! /usr/bin/env python
import rospy
import time
import actionlib

from actionlib.msg import TestFeedback, TestResult, TestAction
from geometry_msgs.msg import Twist
from std_msgs.msg import Empty

class MoveSquareClass(object):
    
  # create messages that are used to publish feedback/result
  _feedback = TestFeedback()
  _result   = TestResult()

  def __init__(self):
    # creates the action server
    self._as = actionlib.SimpleActionServer("move_drone_square_as", TestAction, self.goal_callback, False)
    self._as.start()
    self.ctrl_c = False
    self.rate = rospy.Rate(10)
    
  
  def publish_once_in_cmd_vel(self, cmd):
    """
    This is because publishing in topics sometimes fails teh first time you publish.
    In continuos publishing systems there is no big deal but in systems that publish only
    once it IS very important.
    """
    while not self.ctrl_c:
        connections = self._pub_cmd_vel.get_num_connections()
        if connections > 0:
            self._pub_cmd_vel.publish(cmd)
            rospy.loginfo("Publish in cmd_vel...")
            break
        else:
            self.rate.sleep()
            
  # function that stops the drone from any movement
  def stop_drone(self):
    rospy.loginfo("Stopping...")
    self._move_msg.linear.x = 0.0
    self._move_msg.angular.z = 0.0
    self.publish_once_in_cmd_vel(self._move_msg)
        
  # function that makes the drone turn 90 degrees
  def turn_drone(self):
    rospy.loginfo("Turning...")
    self._move_msg.linear.x = 0.0
    self._move_msg.angular.z = 1.0
    self.publish_once_in_cmd_vel(self._move_msg)
    
  # function that makes the drone move forward
  def move_forward_drone(self):
    rospy.loginfo("Moving forward...")
    self._move_msg.linear.x = 1.0
    self._move_msg.angular.z = 0.0
    self.publish_once_in_cmd_vel(self._move_msg)
    
  def goal_callback(self, goal):
    # this callback is called when the action server is called.
    # this is the function that computes the Fibonacci sequence
    # and returns the sequence to the node that called the action server
    
    # helper variables
    r = rospy.Rate(1)
    success = True
    
    # define the different publishers and messages that will be used
    self._pub_cmd_vel = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
    self._move_msg = Twist()
    self._pub_takeoff = rospy.Publisher('/drone/takeoff', Empty, queue_size=1)
    self._takeoff_msg = Empty()
    self._pub_land = rospy.Publisher('/drone/land', Empty, queue_size=1)
    self._land_msg = Empty()
    
    # make the drone takeoff
    i=0
    while not i == 3:
        self._pub_takeoff.publish(self._takeoff_msg)
        rospy.loginfo('Taking off...')
        time.sleep(1)
        i += 1
    
    # define the seconds to move in each side of the square (which is taken from the goal) and the seconds to turn
    sideSeconds = goal.goal
    turnSeconds = 1.8
    
    i = 0
    for i in xrange(0, 4):
    
      # check that preempt (cancelation) has not been requested by the action client
      if self._as.is_preempt_requested():
        rospy.loginfo('The goal has been cancelled/preempted')
        # the following line, sets the client in preempted state (goal cancelled)
        self._as.set_preempted()
        success = False
        # we end the calculation of the Fibonacci sequence
        break
    
      # Logic that makes the robot move forward and turn
      self.move_forward_drone()
      time.sleep(sideSeconds)
      self.turn_drone()
      time.sleep(turnSeconds)
      
      # build and publish the feedback message
      self._feedback.feedback = i
      self._as.publish_feedback(self._feedback)
      # the sequence is computed at 1 Hz frequency
      r.sleep()
    
    # at this point, either the goal has been achieved (success==true)
    # or the client preempted the goal (success==false)
    # If success, then we publish the final result
    # If not success, we do not publish anything in the result
    if success:
      self._result.result = (sideSeconds*4) + (turnSeconds*4)
      rospy.loginfo('The total seconds it took the drone to perform the square was %i' % self._result.result )
      self._as.set_succeeded(self._result)
        
      # make the drone stop and land
      self.stop_drone()
      i=0
      while not i == 3:
          self._pub_land.publish(self._land_msg)
          rospy.loginfo('Landing...')
          time.sleep(1)
          i += 1
      
if __name__ == '__main__':
  rospy.init_node('move_square')
  MoveSquareClass()
  rospy.spin()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: move_drone_square.py** </p>

如你所见，在上面的代码中，我们通过确定机器人沿正方形每一条边移动的时间（秒钟）来定义正方形是否更大或更小。因此，目标中定义的秒数越多，正方形将越大。这一点可通过多种方法实现，如通过检查里程计数据来识别无人机实际运动的距离等，但这相当复杂，且不是该练习的目的。该练习的目的是学习如何建立一个合适的行为服务器，并与行为的目标、反馈和结果进行交互。

为了测试该代码，你必须首先启动行为服务器 (在上面的Python文件中的定义）。运行下面的命令：

In [ ]:
roslaunch exercise_413 move_drone_saquare.launch

现在，如果运行下面的命令：

In [ ]:
rostopic list | grep move_drone_square

你将看到类似下面的输出：

<img src="../img/move_drone_square_as.png" width="600" />

这意味着你的行为服务器已经启动，做好了接收目标的准备。

现在，向该行为服务器发布一个目标。有两种方法（如行为笔记本第1部分中的那样）可实现这一点：

* 创建一个行为客户端
* 通过行为主题直接发布一个目标

对于该例，我们使用第二种方法。你应发布一个消息至 **move_drone_square_as/goal** 主题，如下：

<img src="../img/goal_square_sol.png" width="800" />

你将看到，无人机运动如下：

<img src="../img/drone_square_sol.gif" width="600" />

最后，查看行为的**feedback** 和 **result**主题，看一下它们是否发布了我们想要的值。

In [ ]:
rostopic echo /move_drone_square_as/feedback

<img src="../img/feedback_square_sol.png" width="600" />

In [ ]:
rostopic echo /move_drone_square_as/result

<img src="../img/result_square_sol.png" width="600" />

## Exercise 4.14解决方案 <p id="SolutionExercise4-14"></p>

<p style="background:#EE9023;color:white;">**Exercise 4.14**</p>

对于该练习，我们假设所使用的功能包为 **exercise_414**, 我们的启动文件为 **action_custom_msg.launch**, Python文件为 **action_custom_msg.py**。

因此 ，对于Exercise 4.14, 我们将创建一个自定义行为消息，该消息使用一个字符串（可以是**UP** 或 **DOWN**）来定义无人机的运动。作为反馈，它将返回一个字符串表示当时无人机所发生的行为。对于结果，它将返回空。

要实现该目的，首先需要做的是在**exercise_414** 功能包中创建一个名为**action**的文件夹。

In [ ]:
roscd exercise_414
mkdir action

接着，在 **action** 文件夹中, 你需要创建一个名为**CustomActionMsg.action**的文件, 内容如下：

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Message File: CustomActionMsg.action** </p>

In [ ]:
string goal
---
---
string feedback

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Message File: CustomActionMsg.action** </p>

如你所见，我们把目标和反馈定义为字符串，而结果则不定义。

接着，按照行为笔记本中描述的方法那样修改 **CMakeLists.txt** 和 **package.xml** 文件。如果你忘记了，查看下面的文件：

<p style="background:#3B8F10;color:white;" id="prg-2-1">**CMakeLists.txt** </p>

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(custom_action)

## Find catkin macros and libraries
## if COMPONENTS list like find_package(catkin REQUIRED COMPONENTS xyz)
## is used, also find other catkin packages
find_package(catkin REQUIRED COMPONENTS
  std_msgs 
  actionlib_msgs
)

## Generate actions in the 'action' folder
add_action_files(
   FILES
   CustomActionMsg.action
 )

## Generate added messages and services with any dependencies listed here
generate_messages(
   DEPENDENCIES
   std_msgs actionlib_msgs
 )

catkin_package(
 CATKIN_DEPENDS rospy
)

## Specify additional locations of header files
## Your package locations should be listed before other locations
# include_directories(include)
include_directories(
  ${catkin_INCLUDE_DIRS}
)

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END CMakeLists.txt** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**package.xml** </p>

In [ ]:
<?xml version="1.0"?>
<package>
  <name>custom_action</name>
  <version>0.0.0</version>
  <description>The my_custom_action_msg_pkg package</description>
  <maintainer email="user@todo.todo">user</maintainer>
  <license>TODO</license>

  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>std_msgs</build_depend>
  <build_depend>actionlib_msgs</build_depend>
  
  <run_depend>rospy</run_depend>

  <export>
  </export>
</package>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END package.xml** </p>

所有这一切完成后，编译功能包，source你将使用的**ALL the webwhells**，以便ROS能够找到这个新的消息。

roscd
cd ..
catkin_make
source devel/setup.bash

最后，查看一下你是否能够找到新的消息：

In [ ]:
rosmsg list | grep CustomActionMsg

你将看到类似下面的输出：

In [ ]:
exercise_414/CustomActionMsgAction
exercise_414/CustomActionMsgActionFeedback
exercise_414/CustomActionMsgActionGoal
exercise_414/CustomActionMsgActionResult
exercise_414/CustomActionMsgFeedback
exercise_414/CustomActionMsgGoal
exercise_414/CustomActionMsgResult

自定义消息创建成功后，你可以继续创建行为服务器来使用这个新的消息。下面是该行为服务器的launch文件和Python文件：

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: action_custom_msg.launch** </p>

In [ ]:
<launch>
    <node pkg="exercise_414" type="action_custom_msg.py" name="action_custom_msg" output="screen" />
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: action_custom_msg.launch** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: action_custom_msg.py** </p>

In [ ]:
#! /usr/bin/env python
import rospy
import time
import actionlib

from custom_action.msg import CustomActionMsgFeedback, CustomActionMsgResult, CustomActionMsgAction
from geometry_msgs.msg import Twist
from std_msgs.msg import Empty

class CustomActionMsgClass(object):
    
  # create messages that are used to publish feedback/result
  _feedback = CustomActionMsgFeedback()
  _result   = CustomActionMsgResult()

  def __init__(self):
    # creates the action server
    self._as = actionlib.SimpleActionServer("action_custom_msg_as", CustomActionMsgAction, self.goal_callback, False)
    self._as.start()

    
  def goal_callback(self, goal):
    # this callback is called when the action server is called.
    # this is the function that computes the Fibonacci sequence
    # and returns the sequence to the node that called the action server
    
    # helper variables
    success = True
    r = rospy.Rate(1)
    
    # define the different publishers and messages that will be used
    self._pub_takeoff = rospy.Publisher('/drone/takeoff', Empty, queue_size=1)
    self._takeoff_msg = Empty()
    self._pub_land = rospy.Publisher('/drone/land', Empty, queue_size=1)
    self._land_msg = Empty()
    
    # Get the goal word: UP or DOWN
    up_or_down = goal.goal
    
    i = 0
    for i in xrange(0, 4):
    
      # check that preempt (cancelation) has not been requested by the action client
      if self._as.is_preempt_requested():
        rospy.loginfo('The goal has been cancelled/preempted')
        # the following line, sets the client in preempted state (goal cancelled)
        self._as.set_preempted()
        success = False
        # we end the calculation of the Fibonacci sequence
        break
    
      # Logic that makes the robot move UP or DOWN
      if up_or_down == 'UP':
        
        self._pub_takeoff.publish(self._takeoff_msg)
        self._feedback.feedback = 'Going UP'
        self._as.publish_feedback(self._feedback)
    
      if up_or_down == 'DOWN':
        
        self._pub_land.publish(self._land_msg)
        self._feedback.feedback = 'Going DOWN'
        self._as.publish_feedback(self._feedback)
    
      # the sequence is computed at 1 Hz frequency
      r.sleep()
    
    # at this point, either the goal has been achieved (success==true)
    # or the client preempted the goal (success==false)
    # If success, then we publish the final result
    # If not success, we do not publish anything in the result
    if success:
      self._result = Empty()
      self._as.set_succeeded(self._result)
      
      
if __name__ == '__main__':
  rospy.init_node('action_custom_msg')
  CustomActionMsgClass()
  rospy.spin()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: action_custom_msg.py** </p>

如你所见，在上面的代码中，为了使无人机向上或向下运动，我们使用 **/drone/takeoff** 和 **/drone/land** 主题，该主题将使无人机移动至大约1米的高度。你也可以使用 **/cmd_vel** 主题，这取决于你自己。

为了测试该代码，你必须首先启动行为服务器(上面的Python文件中进行了定义)。运行下面的命令：

In [ ]:
roslaunch exercise_414 action_custom_msg.launch

现在，如果运行下面的命令：

In [ ]:
rostopic list | grep action_custom_msg

你将会看到类似下面的结果：

<img src="../img/action_custom_msg_as.png" width="600" />

这意味着你的行为服务器已经启动，做好了接收目标的准备。

现在，让我们向该行为服务器发布一个主题。有两种方法（如行为笔记本第1部分中的那样）可实现这一点：

* 创建一个行为客户端
* 通过行为主题直接发布一个目标

对于该例，我们使用第二种方法。你应发布一个消息至 **action_custom_msg_as/goal**主题，如下：

<img src="../img/action_custom_msg_as_goal.png" width="800" />

你将看到，无人机运动如下：

<img src="../img/up_or_down1.gif" width="600" />

<img src="../img/up_or_down2.gif" width="600" />

最后，查看行为的**feedback** 和 **result**主题，看一下它们是否发布了我们想要的值。

In [ ]:
rostopic echo /action_custom_msg_as/feedback

<img src="../img/action_custom_msg_as_feedback.png" width="600" />

<img src="../img/action_custom_msg_as_feedback2.png" width="600" />

该例中，没有结果要显示。